<div class="title"> Modern Bayesian methods: principles and practice</div>

<hr/>

## Vinayak Rao, Purdue University
### Jan 4, 2021

Material at $\mathtt{github.com/varao/ds3\_bayesian}$




In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import pystan
import arviz as az
from matplotlib import pyplot as plt
import numdifftools as nd    # Optional

In [ ]:
# Data from https://github.com/ImperialCollegeLondon/covid19model
covid_agg = pd.read_csv('covid_agg.csv')
covid_agg.head()

In [ ]:
covid_agg['rate'] = covid_agg.cases/covid_agg.popData2018
covid_agg.rate.hist()

### Modeling the infection rate



### A brief intro to Stan (from the Stan website)



Stan is a state-of-the-art platform for statistical modeling and high-performance statistical computation.

Users specify log density functions in Stan’s probabilistic programming language and get:

+ full Bayesian statistical inference with MCMC sampling (NUTS, HMC)

+ approximate Bayesian inference with variational inference (ADVI)

+ penalized maximum likelihood estimation with optimization (L-BFGS)

Stan interfaces with the most popular data analysis languages (R, Python, shell, MATLAB, Julia, Stata) 

Stan User guide: https://mc-stan.org/docs/2_25/stan-users-guide/index.html

Stan reference manual: https://mc-stan.org/docs/2_25/reference-manual/index.html

PyStan: https://pystan.readthedocs.io/en/latest/api.html


In [ ]:
model1_code = """
data {
    int<lower=0> N; // number of countries
    vector[N] x; // proportion of cases
}

parameters {
    real<lower=0> a;
    real<lower=0> b;

}

model {
    a ~ exponential(.0001);
    b ~ exponential(.0001);
    for(i in 1:N) {
      x[i] ~ beta(a,b);
    }
}

"""
sm1 = pystan.StanModel(model_code=model1_code)

In [ ]:
model1_data = {'N': len(covid_agg.rate), 'x': covid_agg.rate}
fit = sm.sampling(data=model1_data, iter=5000, chains=1)
az.plot_density(fit);

print(fit);   #pd.DataFrame(fit.extract())

### Exercise

+ Is this a good fit to the data? How would you tell?

+ What is the distribution of first observation?

Instead of the infection rate, let us model the log-odds

For a probability of success $p$, the log odds are $\log(\frac{p}{1-p})$

In [ ]:
covid_agg['logodds'] = np.log(covid_agg.rate/(1-covid_agg.rate))
covid_agg.logodds.hist(bins=20)

In [ ]:
model2_code = """
data {
    int<lower=0> N; // number of countries
    real<lower=0> sigma; 
    vector[N] x; // proportion of cases
}

parameters {
    real mu;
}

model {
    mu ~ normal(0, 10);
    for(i in 1:N) {
      x[i] ~ normal(mu,sigma);
    }
}
    
generated quantities {
    real mn;
    real mx; 
    real std;
    {
      vector[N] x_rep;
      for(j in 1:N) {
        x_rep[j] <- normal_rng(mu, sigma);
      }
      std = sd(x_rep);
      mn = min(x_rep);
      mx = max(x_rep);
    }
}
"""
sm2 = pystan.StanModel(model_code=model2_code)

In [ ]:
model2_data = {'N': len(covid_agg.logodds), 'sigma':11, 'x': covid_agg.logodds}
fit = sm2.sampling(data=model2_data, iter=5000, chains=1)

az.plot_density(fit);

print(fit);

### Model checking

Is the previous model a good fit of the data?

How can you quantify this?

+ Cross-validation

+ Posterior predictive checks



In [ ]:
model3_code = """
data {
    int<lower=0> N; // number of countries
    vector[N] x; // proportion of cases
}

parameters {
    real mu;
    real<lower=0> sigma; 

}

model {
    mu ~ normal(0, 10);
    sigma ~ gamma(.01,.01);
    for(i in 1:N) {
      x[i] ~ normal(mu,sigma);
    }
}
"""
sm3 = pystan.StanModel(model_code=model3_code)

In [ ]:
model3_data = {'N': len(covid_agg.logodds), 'x': covid_agg.logodds}

fit = sm3.sampling(data=model3_data, iter=5000, warmup=100, chains=1)
az.plot_density(fit)

### Exercises
+ Can we place a prior on the prior mean?

+ What properties of the data might the previous model fail to capture?

+ Modify the code to quantify this

### Hierarchical Bayes

In [ ]:
model_hier_code = """
data {
    int<lower=0> N; // number of countries
    int<lower = 1> C; // number of continents

    vector[N] x; // log-odds of proportion of cases
    int<lower=1, upper=C> cont[N];  // continent
}

parameters {
    real mu0;
    real<lower=0> sigma0; 
    
    vector[C] mu_cont;
    real<lower=0> sigma; 
}

model {
    mu0 ~ normal([0,0], 10);
    sigma0 ~ gamma(.01,.01);
    sigma ~ gamma(.01,.01);

    for(i in 1:C) {
      mu_cont[i] ~ normal(mu0,sigma0);
    }
    x ~ normal(mu_cont[cont],sigma);
}
"""
sm_hier = pystan.StanModel(model_code=model_hier_code)

In [ ]:
model_hier_data = {'N': len(covid_agg.logodds), 'C': max(covid_agg.cont), 'x': covid_agg.logodds, 
               'cont': covid_agg.cont}
fit = sm_hier.sampling(data=model_hier_data, iter=5000, warmup=1000, chains=1)
az.plot_density(fit)

### Exercise

+ What are different ways you can extend the hierarchical model above to allow more flexibility?
+ How do you expect the parameter estimates to differ from the situation where each group is modeled independently?
+ Write Stan code for the latter case


### Markov chain Monte Carlo (MCMC)




### The Titanic dataset



Around 800 measurements including:

<ul>
<li> survival ($y$): Survival (0 = No; 1 = Yes)
<li> pclass ($x_1$): Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
<li> sex ($x_2$): Sex (1 = female, 0 = male)
<li> age ($x_3$): Age
<li> ticket ($x_3$): Ticket Number
<li> fare ($x_4$): Passenger Fare
</ul>




In [ ]:
titanic = pd.read_csv('titanic.csv')
titanic.head()

In [ ]:
model_code = """

data {
  int<lower=0> N; // number of obs
  int<lower=0> P; // number of predictors
  int<lower=0,upper=1> y[N]; // outcomes
  matrix[N, P] x; // predictor variables
}
parameters {
  vector[P] theta; // theta coefficients
}
model {
  vector[N] mu; 
  theta ~ normal(0, 100);   // cauchy(0,10) is what Gelman et al recommend
  mu <- x*theta;
  for (n in 1:N) mu[n] <- Phi(mu[n]);
  y ~ bernoulli(mu);
}
"""
sm_probit = pystan.StanModel(model_code=model_code)

In [ ]:
titanic_data = {'N': len(titanic.survived), 'P': titanic.shape[1]-1, 
                'x': titanic.drop('survived',axis=1), 'y': titanic.survived}
fit = sm_probit.sampling(data=titanic_data, iter=5000, warmup=1000, chains=1)

In [ ]:
az.plot_density(fit)
print(fit)

In [ ]:
def probit_loglik(theta, x,y):
    mu = x @ theta
    
    a = y*sp.stats.norm(0,1).logcdf(mu)
    b =  (1-y)*np.log(1-sp.stats.norm(0,1).cdf(mu))
    return sum(a+b)-(theta.T @ theta)/(2*100)

def probit_mh(niter, M, data):
    N, P, x, y = data['N'], data['P'], data['x'], data['y']
    y = y[:,np.newaxis]
    
    theta = np.zeros([P, niter])
    
    for i in np.arange(1, niter):
        prop = np.random.multivariate_normal(theta[:,i-1], M)
        prop = prop[:,np.newaxis]
        #print(theta[i,].T)

        #print(probit_loglik(theta[i,].T,x,y))

        if np.log(np.random.uniform()) < (probit_loglik(prop,x,y) - probit_loglik(theta[:,[i-i]],x,y)):
            theta[:,[i]] = prop
        else:
            theta[:,[i]] = theta[:,[i-1]]
    return theta.T
    
M = np.identity(titanic_data['P'])
M = np.linalg.inv(iM)
rslt = probit_mh(1000,M*5,titanic_data)


In [ ]:
rslt.mean(0), rslt.std(0)
#jnk = rslt[1,]
#jnk[:,np.newaxis]
plt.plot(rslt[:,0])

In [ ]:
[az.ess(rslt[:,[i]].T) for i in range(6) ]

### Exercise

+ Play around with the M above. What gives you higher effective sample size (ESS)? How does it compare to Stan's results?
+ How would you debug your sampler to make sure it is working?
+ Does a high ESS mean a better sampler?
+ Does a high acceptance rate mean a better sampler?
+ Can the proposal variance also depend on the current location?

### A brief look at Bayesian nonparametrics



In [ ]:
def my_kernel(X,s,l):
    n = len(X)
    p = 2
    D = np.zeros([n,n])

    for i in range(n):
        D[i,] = np.abs((X[i] - X)) ** p
    
    return s*np.exp(-0.5*D/l) + 1e-5*np.eye(n)

X = np.arange(0,10,step=.01)
cv = my_kernel(X,3,1)

plt.plot(X, np.random.multivariate_normal(X*0,cv))


__Exercise__: play around with the parameters s,l,p of the the kernel. Note down your conclusions

In [ ]:
model_code = """
data {
  int<lower=1> N;
  real x[N];
  vector[N] y;
}

parameters {
  real<lower=0> rho;
  real<lower=0> alpha;
  real<lower=0> sigma;
  vector[N] eta;
}

transformed parameters {
  vector[N] f;
  {
    matrix[N, N] L_K;
    matrix[N, N] K = cov_exp_quad(x, alpha, rho);

    // diagonal elements
    for (n in 1:N)
      K[n, n] = K[n, n] + 1e-4;

    L_K = cholesky_decompose(K);
    f = L_K * eta;
  }
}

model {
  real sq_sigma = square(sigma);


  eta ~ std_normal();

  rho ~ inv_gamma(5, 5);
  alpha ~ std_normal();
  sigma ~ std_normal();

  y ~ normal(f, sq_sigma);
}
"""

sm_gp = pystan.StanModel(model_code=model_code)

In [ ]:
model_gp_data = {'N': len(covid_agg.rate), 'y': np.log(covid_agg.cases), 'x': np.log(covid_agg.popData2018), 
               'cont': covid_agg.cont}
fit = sm_gp.sampling(data=model_gp_data, iter=1000,  chains=1)
#az.plot_density(fit)

In [ ]:
tmp = fit.extract('f')['f']
mn = tmp.mean(axis=0)
sd = tmp.std(axis=0)

rslt = pd.DataFrame({'x': model_gp_data['x'], 'mn':mn, 'uci': mn+3*sd, 'lci': mn-3*sd})
rslt =  rslt.sort_values(by='x')

plt.plot(rslt.x,rslt.mn)
plt.plot(rslt.x,rslt.uci)
plt.plot(rslt.x,rslt.lci)
plt.scatter(rslt.x, model_gp_data['y'])
